# Word2Vec

▶️ Utiliser [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) ≥ 4.0!

In [ ]:
!pip freeze | grep gensim


In [ ]:
!pip freeze | grep tensorflow


# Données

Keras met à disposition plusieurs datasets, on va utiliser l'IMDB dataset 🎬:
- Chaque document est une ***review d'un film***. 
- Chaque review est liée à un score donnée par le spectateur

In [ ]:
###########################################
### Charger les données ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)

    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)

        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]

        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]

    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)


<img src="layers_embedding.png" width="400px" />

L'entraînement de l'embedding ajoute de la complexité et augmente le temps total d'entraînement

<img src="word2vec_representation.png" width="400px" />

On va tester l'embedding avec Word2Vec.

L'inconvenient est que l'embedding n'est pas entrîné specifiquement pour notre tâche,  MAIS, il y a des avantages:
- rapidité d'embedding
- l'embedding est quand même representatif
- la convergence du RNN sera plus rapide et facile

# Embedding  avec Word2Vec

![Embedding](word_embedding.png)

❓ Faire de l'embedding sur le dataset d'entraînement avec word2vec

In [ ]:
from gensim.models import Word2Vec


❓ Regarder la representation de quelques mots 

❓ Quelle est la dimension de l'embedding?

🧐 Comment juger la qualité de l'embedding?

💡 Regarder si des mots proches semantiquement sont proches dans l'espace vectoriel

👉 [**`Word2Vec.wv.most_similar`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) est une methode qui retourne les mots les plus proches dans l'espace vectoriel à l'input donné

❓ Tester la methode `most_similar` en quelques mots

La méthode [**`similar_by_vector`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) existe aussi

# Opérations vectorielles

$$W2V(good) - W2V(bad)$$

❓ Réaliser cette opération et afficher le résultat

In [ ]:
wv['good'] - wv['bad']


Imaginons que:

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

Ce qui est equivalent à:

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ Réaliser cette operation et sauvegarder le résultat dans une variable

❓ Utiliser `similar_by_vector` pour regarder les mots les plus proches dans l'espace vectoriel

❓ Essayer 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

ou 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ Si les résultats sont pas parfaites faut pas oublier la taille du corpus

# Hyperparametres

❓ `vector_size` correspond à la dimension de l'embedding. Entraîner un nouveau modèle `word2vec_2` avec `X_train` mais avec `vector_size` différent.

❓ Utiliser **`Word2Vec.wv.key_to_index`** pour afficher la taille du vocabulaire appris. Compare avec les mots en `X_train`.

Il y a une différence importante entre le nombre de mots dans les phrases d'entraînement et dans le vocabulaire de Word2Vec, même si ce dernier a été entraîné sur l'ensemble des phrases d'entraînement. La raison vient du deuxième hyperparamètre important de Word2Vec : `min_count`. 

`min_count` est un entier qui vous indique combien d'occurrences un mot donné doit avoir pour être appris dans l'espace d'intégration. Par exemple, disons que le mot "movie" apparaît 1000 fois dans le corpus et "simba" seulement 2 fois. Si `min_count=3`, le mot "simba" sera ignoré pendant l'apprentissage.

L'objectif est d'apprendre une représentation des mots qui sont suffisamment présents dans le corpus pour avoir une représentation intégrée robuste.

Traduit avec www.DeepL.com/Translator (version gratuite)

❓ Apprenez un nouveau modèle `word2vec_3` avec un `min_count` supérieur à 5 (qui est la valeur par défaut) et un `word2vec_4` avec un `min_count` inférieur à 5, puis comparez la taille du vocabulaire pour tous les différents `word2vec` que vous avez appris (vous pouvez choisir n'importe quelle `vector_size`).

Rappelons que Word2Vec dispose d'un réseau neuronal interne qui est optimisé sur la base de certaines prédictions. Ces prédictions correspondent en fait à la prédiction d'un mot en fonction des mots environnants. Les mots environnants se trouvent dans une "fenêtre" qui correspond au nombre de mots pris en compte. Vous pouvez entraîner Word2Vec avec différentes tailles de "fenêtres".

❓ Entraîner un nouveau modèle `word2vec_5` avec une `fenêtre` différente de la précédente (la valeur par défaut est 5).

Les arguments que vous avez vu (`vector_size`, `min_count` et `window`) sont généralement ceux avec lesquels vous devriez commencer à jouer pour obtenir de meilleures performances pour votre modèle.

Mais vous pouvez aussi regarder d'autres arguments dans la [**📚 Documentation - gensim.models.word2vec.Text8Corpus**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)

# Convertir nos ensembles d'entraînement et de test en ensembles de données prêts pour le RNN

Rappelez-vous que `Word2Vec` est la première étape du processus global d'introduction d'une telle représentation dans un RNN, comme illustré ici :

<img src="word2vec_representation.png" width="400px" />



Passons maintenant à l'étape 2 en convertissant les données d'entraînement et de test en leur représentation vectorielle afin qu'elles soient prêtes à être introduites dans les RNN.

❓ Maintenant, écrire une fonction qui, étant donné une phrase, renvoie une matrice qui correspond à l'embedding de la phrase complète, ce qui signifie que vous devez intégrer chaque mot l'un après l'autre et concaténer le résultat pour obtenir une matrice 2D (assurez-vous que votre résultat est un tableau NumPy).

❗ **Remarque** ❗ Vous remarquerez probablement que certains mots que vous essayez de convertir provoquent des erreurs car ils n'appartiennent pas au dictionnaire :

- Pour le test ceci est normal, quelques mots ne seront pas presentes dans les données d'entraînement et donc l'embedding est inconnu.
- Pour le train le `min_count` fait que pas tous les mots aient une representation vectorielle.

Dans tous les cas, il suffit de sauter les mots manquants ici.

In [ ]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):

    pass


### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))


❓ Écrire une fonction qui, étant donné une liste de phrases (chaque phrase étant une liste de mots/chaînes), renvoie une liste de phrases intégrées (chaque phrase est une matrice). Appliquez cette fonction aux phrases de formation et de test

💡 _Piste_ : Utiliser la fonction précédente `embed_sentence`

In [ ]:
def embedding(word2vec, sentences):

    pass


X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)


❓ Afin d'avoir des données prêtes à l'emploi, n'oubliez pas le padding de vos séquences afin d'avoir des tenseurs qui peuvent être divisés en lots (de `batch_size`) pendant l'optimisation. Stockez les valeurs paddées dans `X_train_pad` et `X_test_pad`. N'oubliez pas les arguments importants du padding

In [ ]:

from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_train, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test, dtype='float32', padding='post')


assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)
